### API Calls



In [70]:
import configparser

config = configparser.ConfigParser()
config.read(r'C:\Users\GisselleMaira\test.ini')

['C:\\Users\\GisselleMaira\\test.ini']

In [71]:
import json
import os
import urllib.request
import requests
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime, timedelta
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.relative_locator import locate_with
# from bs4 import BeautifulSoup
import shutil
import glob
import os.path
import time

In [72]:
import openpyxl
import xlsxwriter
from email.message import EmailMessage
# import smptlib
import win32com.client


In [73]:
acct = config['api']['account']
token = config['api']['token']

payload={}
headers = {
  'Harvest-Account-Id': acct,
  'Authorization': token
}

In [139]:
# budget data
url = f"https://api.harvestapp.com/v2/reports/project_budget?is_active=true"

response = requests.request("GET", url, headers=headers, data=payload).text

data_json  = json.loads(response)['results']  

In [6]:
# assignement data
# url2 = f"https://api.harvestapp.com/v2/users/me/project_assignments"

# resp2 = requests.request("GET", url2, headers=headers, data=payload).text

# data_json2  = json.loads(resp2)['project_assignments']  

In [14]:
# user data
url3 = f"https://api.harvestapp.com/v2/users?is_active=true"
resp3 = requests.request("GET", url3, headers=headers, data=payload).text

data_json3  = json.loads(resp3)['users']
# data_json3

In [12]:
# roles
# url4 = f"https://api.harvestapp.com/v2/roles"
# resp4 = requests.request("GET", url4, headers=headers, data=payload).text
# data_json4  = json.loads(resp4)['roles']    

In [140]:
budget = pd.json_normalize(data_json)
# assign = pd.json_normalize(data_json2)
users = pd.json_normalize(data_json3)
# roles = pd.json_normalize(data_json4)

In [16]:
# data_json2_task = pd.json_normalize(
#     data_json2, meta = ['id' ],  \
#     record_path = 'task_assignments', record_prefix='task_', errors = 'ignore')

# assign = pd.json_normalize(data_json2)

# assign.columns = assign.columns.str.replace('.', '_')

# assign = assign.drop(columns=['task_assignments'])

In [17]:

# assign_fin = pd.merge(assign, data_json2_task, on = ['id'], how='left')
# assign_fin.head()

In [18]:
# writer = pd.ExcelWriter(r"C:\Users\GisselleMaira\Desktop\users.xlsx", engine='xlsxwriter')

# users.to_excel(writer, sheet_name='users', index = False)
# assign_fin.to_excel(writer, sheet_name='assign', index = False)
# budget.to_excel(writer, sheet_name='budget', index = False)
# roles.to_excel(writer, sheet_name='roles', index = False)

# writer.save()
# incomplete list
# users[users.is_project_manager == True]

In [75]:
# filter out non billable
projects = budget[~budget.client_name.isin(['0000_Spinnaker', '0070_Dotted Line'])]
projects = projects[['project_name','client_name', 'budget','budget_spent','budget_remaining']][projects.is_active == True]
# budget

In [203]:
users2 = users.explode('roles')

# isolate emails and filter out FP - they can have multiple assignments
FPemail = users2[users2['roles'].isin(['Flying Phase'])]['email']
nonFP = users2[~users2.email.isin(FPemail)]
FP = users2[users2.email.isin(FPemail)]

# get advisor/consultants only
FP2 = FP[FP['roles'].isin(['Advisor', 'Consultant'])][['id', 'first_name', 'last_name', 'email', 'default_hourly_rate', 'cost_rate', 'roles']]
BU2 = users2[users2['roles'].isin(['Flying Phase'])][['id', 'email', 'roles']]
FP3 = pd.merge(FP2, BU2 , on = ['id', 'email'], how='left')

nonFP2 = nonFP[nonFP['roles'].isin(['Advisor', 'Consultant'])][['id', 'first_name', 'last_name', 'email', 'default_hourly_rate', 'cost_rate', 'roles']]
BU1 = users2[users2['roles'].isin(['CCOM', 'Risk', 'Data and Analytics'])][['id', 'email', 'roles']]
pd.merge(users3, BU , on = ['id', 'email'], how='left')
nonFP3 = pd.merge(nonFP2, BU1 , on = ['id', 'email'], how='left')


all_users = pd.concat([nonFP3,FP3], axis=0)
all_users.rename(columns={'roles_x': 'role', 'roles_y': 'BusUnit'}, inplace=True)

In [204]:
all_users.columns

Index(['id', 'first_name', 'last_name', 'email', 'default_hourly_rate',
       'cost_rate', 'role', 'BusUnit'],
      dtype='object')

## Temporary data to test email.

In [20]:
# sample data for project manager data (placeholder only )

data = pd.DataFrame({'User_id': [3728213]*10 + [3697393]*5 + [0] + [123] + [456]})
# data
df = pd.DataFrame(data, columns = ['User_id'])

In [21]:

proj_2 = pd.concat([projects, df], axis=1)
# proj_2.head()

In [22]:
# add email to project list
final_proj = pd.merge(proj_2, users[['id', 'email']] , left_on = ['User_id'], right_on = ['id'], how='left')
# final_proj.head()

In [24]:
final_proj.replace('Brenton.Eska@Spinnakerconsultinggroup.com', 'jessiezz@hotmail.com', inplace = True)
# final_proj.head(15)

## Temporary section over.  Edit below prior to automating.

In [26]:
email_list = ['Gisselle.Maira@spinnakerconsultinggroup.com','gisselle.maira@spinnakerconsultinggroup.com']
# email_list=final_proj.email[~final_proj['email'].isnull()].unique().tolist()
# email_list

['Gisselle.Maira@spinnakerconsultinggroup.com',
 'gisselle.maira@spinnakerconsultinggroup.com']

In [25]:
# create email_num to groupby
final_proj['email_num'] = np.where(final_proj['email']!= 'jessiezz@hotmail.com', 1, 2)
final_proj.replace('jessiezz@hotmail.com', 'gisselle.maira@spinnakerconsultinggroup.com', inplace = True)

# automated list of numbers to be merged with email_list
# final_proj['email_num'] = [*range(1, len(email_list) + 1, 1)]

# count non null emails and group data
# end = len(final_proj.email[~final_proj['email'].isnull()].unique())

In [ ]:
# list of alphabetical emails
# email_list=final_proj.email[~final_proj['email'].isnull()].unique().tolist()

num = final_proj['email_num'].unique()
list(num)

## Edit final output 

In [ ]:
pd.options.display.float_format = '${:,.2f}'.format

# final_proj.set_index(keys=['email'], drop=True,inplace=True)
final_proj.sort_values(by='email', axis=0, inplace=True)
final_proj.drop(['User_id', 'id'],1, inplace = True)
final_proj.dropna(subset=['project_name', 'email'], inplace = True)
# final_proj.head()



In [ ]:
# grouped = final_proj.groupby(['email'])
grouped = final_proj.groupby(['email_num'])
l_grouped = list(grouped)

In [ ]:
# l_grouped['email_num' == 1][1][['project_name', 'client_name', 'budget', 'budget_spent', 'budget_remaining']]
# l_grouped[1][1] 
# l_grouped[0][1]
# d = dict(tuple(final_proj.groupby('email')))
# d['Gisselle.Maira@Spinnakerconsultinggroup.com']
# final_proj.reset_index(drop=True, inplace=True)
# final_proj.index

# list of alphabetical emails


In [ ]:
# final_proj[final_proj['email_num']==(1)][['project_name', 'client_name', 'budget', 'budget_spent', 'budget_remaining']]

## Email Creation

In [228]:
# URLs = pd.read_excel(r"C:\Users\GisselleMaira\Desktop\ProjectURLs.xlsx")
# URLs.URL.count()

# # URLs
# for idx in URLs.index:
#    print(URLs['URL'][idx])
   ## Do your selenium tasks here for opening and logging into the fetched link.
    
    https://spinnakerconsultinggroup.harvestapp.com/projects?filter=active

18

In [230]:
# for range():
#     if url != "":
#         print(url)
#         sleep(2)
# driver.quit()


SyntaxError: invalid syntax (<ipython-input-230-625767e0aafa>, line 2)

In [227]:
# URLs.URL

# with open("links.txt") as url_file:
#     for line in url_file.readlines():
#         url = line.strip()
#         if url != "":        
#             driver.get(url)
#             host = urlparse(url).hostname
#             driver.get_screenshot_as_file("{}.png".format(host))            
#             sleep(2)

# driver.quit()
# print("end...")

0     https://spinnakerconsultinggroup.harvestapp.co...
1     https://spinnakerconsultinggroup.harvestapp.co...
2     https://spinnakerconsultinggroup.harvestapp.co...
3     https://spinnakerconsultinggroup.harvestapp.co...
4     https://spinnakerconsultinggroup.harvestapp.co...
5     https://spinnakerconsultinggroup.harvestapp.co...
6     https://spinnakerconsultinggroup.harvestapp.co...
7     https://spinnakerconsultinggroup.harvestapp.co...
8     https://spinnakerconsultinggroup.harvestapp.co...
9     https://spinnakerconsultinggroup.harvestapp.co...
10    https://spinnakerconsultinggroup.harvestapp.co...
11    https://spinnakerconsultinggroup.harvestapp.co...
12    https://spinnakerconsultinggroup.harvestapp.co...
13    https://spinnakerconsultinggroup.harvestapp.co...
14    https://spinnakerconsultinggroup.harvestapp.co...
15    https://spinnakerconsultinggroup.harvestapp.co...
16    https://spinnakerconsultinggroup.harvestapp.co...
17    https://spinnakerconsultinggroup.harvestap

In [ ]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

In [ ]:

for i in range(len(num)):
#     print(i+1)
#     dfa= final_proj[final_proj['email']==i]
    dfa = final_proj[final_proj['email_num']==(i+1)][['project_name', 'client_name', 'budget', 'budget_spent', 'budget_remaining']]
#     print(dfa)
    mail = outlook.CreateItem(0)
    mail.To = email_list[i]
    mail.Subject = 'Budget updates'
    mail.HTMLBody = '''<h3>Please find budget updates below.</h3>
                     {}'''.format(dfa.to_html())
    print(f"Email {} sent. ")
    mail.Send()

In [ ]:
# splits = [projects.loc[[i]] for i in projects.index]
# splits[2]
# user_assign = pd.merge(assign, users, on = 'id', how='left')

In [233]:
import os
import sys


In [234]:
os.path.dirname(sys.executable)

'C:\\Users\\GisselleMaira\\anaconda3'